# Algebra for Reinforcement Learning network control

However, continues control problems are hard and open problem in Reinforcement Learning (RL), often requires to implement Artificial Neural Networks (ANN) [2]. In this framework we have decided to stick to a discrete variant of this problem. We consider all the network states over a finite fields or Galois Fields ($GF(p^k)$) with _just limited amount_ of $(p^k)^n$ possible states.

Additionally, we find it very handy to store network states as $(p^k)$ base numbers along with the state vectors. For smaller bases a significant space is saved.

Unlike for a traditional continues problem over a real numbers, using $GF(p^k)$ and $(p^k)$ based numbers put some constrains on using classical "network algebra" of adjacency matrix, we need to adjust our computation accordingly. 

_p is a prime, k - prime factor, n - number of nodes, for further notation see [notation notebook](./00_notation.ipynb)_

In [1]:
import os
import sys
import numpy as np

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from network_control_rl_framework.algebra import BaseNumber, FiniteField
from network_control_rl_framework.network import Network, calculate_next_state, calculate_next_state_base_number

## Computations over a finite field
Finite fields or Galois fields $GF(p^k)$ are fields with finite number of elements. The number of elements $p^k$ are called order, $p$ has to be prime number and $k$ is any positive integer.

Computations over finite fields with prime order are trivial, they are $mod\ p$. While for $k > 1$ they are not so obvious and theory of polyninomial is required. Fortunately, there is a nice link between binary numbers and $GF(2^K)$, so any computations can be _easily_ achieved [1].

In [2]:
# Example of 2 and 1 over GF(3)
a = FiniteField(2, 3)  
b = FiniteField(1, 3)
print(f"a + b = {(a + b).a}")
print(f"a - b = {(a - b).a}")
print(f"a * b = {(a * b).a}")
print(f"a / b = {(a / b).a}")     

a + b = 0
a - b = 1
a * b = 2
a / b = 2


In [3]:
# Example of 2 and 3 over GF(2^2)
a = FiniteField(2, 4)  
b = FiniteField(3, 4)
print(f"a = {a}")
print(f"b = {b}\n")

print(f"a + b = {(a + b).a}")
print(f"a - b = {(a - b).a}")
print(f"a * b = {(a * b).a}")
print(f"a / b = {(a / b).a}") 

a = BinaryFiniteField(2, 2) GF(2^2) ≃ Z_2 / <x^2+x^1+1>
b = BinaryFiniteField(3, 2) GF(2^2) ≃ Z_2 / <x^2+x^1+1>

a + b = 1
a - b = 2
a * b = 1
a / b = 3


## $p^k$ - base number
The radix or base is the number of unique digits used in positional number systems. They are counted from 0 to $p^k - 1$ (if $p^k - 1$ is larger than 9, letters are used - _a, b, c, e, f_). In this framework, only 2-16 bases are used.

Subscript number denotes the base e.g. $1101_3$ = $37_{10}$ means number $1101$ in base $3$ is the same as number $37$ in base $10$.

In the network control, base number denotes vectors, e.g. $x = [1,1,0,1]$ is a state vector for a network with $n=4$ over base 3, therefore, for simplicity $x$ can be turn into $37_{10}$.

In [4]:
# Simple operation over two state vector of n=4 in base 3
a_3 = BaseNumber(4, q=3)
a_3.from_array(np.array([0, 1, 2, 1], dtype=np.int8))
b_3 = BaseNumber(4, q=3)
b_3.from_array(np.array([0, 0, 0, 1], dtype=np.int8))

print(f"a_3 = {a_3.a}_10")
print(f"b_3 = {b_3.a}_10\n")

print(f"a_3 + b_3 = {(a_3 + b_3).a}_10")

a_3 = 16_10
b_3 = 1_10

a_3 + b_3 = 17_10


## Networks and states' envolution
_For network control theory see [network control theory notebook](./01_network_control.ipynb)_  

Putting all together, network states are denoted as base numbers over a finite field, therefore, all the computation needs to be align with it.

In the following example, there is a simple line network of $n=4$ where we send signals into the first - source node and let the signal be propagated into the rest of the network. According to the time invariant discrete control problem given by: 

$x_{t+1} = Ax_t + Bu_{t+1}$

At every time step, the first - soruce node will recivieve a signal $u_{t+1} = 1_3$.

In [5]:
line_network = Network()
line_network.from_edges([(0, 1), (1, 2), (2, 3)])

x = BaseNumber(line_network.nodes, q=3)
x.from_array(np.array([0, 1, 2, 1], dtype=np.int8))

signal = BaseNumber(1, q=3)
signal.from_array(np.array([1], dtype=np.int8))
input_matrix = {0: 0}  # driver node is the node 1

In [6]:
print(f"x_{0}={x.to_array()} ({x.a}_10)")
for t in range(1, 5):
    x = calculate_next_state_base_number(line_network, x, signal, input_matrix)
    print(f"x_{t}={x.to_array()} ({x.a}_10)")

x_0=[0 1 2 1] (16_10)
x_1=[1 0 1 2] (32_10)
x_2=[1 1 0 1] (37_10)
x_3=[1 1 1 0] (39_10)
x_4=[1 1 1 1] (40_10)


## References
[1] John Kerl, (2004), "Computation in finite fields", https://johnkerl.org/doc/ffcomp.pdf  
[2] Volodymyr Mnih1, Koray Kavukcuoglu, David Silver, et.al. (2015), "Human-level control through deep reinforcement
learning", https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf  